In [ ]:
%%capture
!pip install sentence_transformers

In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer, util
from datasets import load_dataset

# calculating average text embeddings of sample questions for each domain

In [ ]:
checkpoint = 'sentence-transformers/paraphrase-MiniLM-L6-v2'
dataset_path = 'biomed_dataset' # sample questions from domains dataset

In [ ]:
model = SentenceTransformer(checkpoint)

In [ ]:
dataset = load_dataset(dataset_path, split='train')
bio = dataset.filter(lambda example: example['domain'] == 'bio')
fin = dataset.filter(lambda example: example['domain'] == 'fin')
legal = dataset.filter(lambda example: example['domain'] == 'legal')

In [ ]:
bio_embedding = model.encode(bio['question'])
fin_embedding = model.encode(fin['question'])
legal_embedding = model.encode(legal['question'])
bio_avg = np.mean(m_embedding, axis=0)
fin_avg = np.mean(f_embedding, axis=0)
legal_avg = np.mean(l_embedding, axis=0)

In [ ]:
with open('bio.npy', 'wb') as f:
  np.save(f, bio_avg)
with open('fin.npy', 'wb') as f:
  np.save(f, fin_avg)
with open('legal.npy', 'wb') as f:
  np.save(f, legal_avg)

# test domain selection accuracy

In [ ]:
label = 0 # 0 for bio, 1 for finance, 2 for legal
test_domain = 'bio' # 'bio', 'fin' or 'legal'
test_size = 100 # number of test questions

with open("bio-avg.npy", 'rb') as f:
  bio_avg = np.load(f)
with open("fin-avg.npy", 'rb') as f:
  fin_avg = np.load(f)
with open("legal-avg.npy", 'rb') as f:
  legal_avg = np.load(f)

In [ ]:
test = load_dataset(dataset_path, split='test')
sub_test = test.filter(lambda example: example['domain'] == test_domain)
sub_test = sub_test.select(range(test_size))

In [ ]:
def select_domain(q, model, bio_avg, fin_avg, legal_avg):
  embedding = model.encode(q)
  bio = [util.cos_sim(embedding, bio_avg).numpy()[0][0]]
  fin = [util.cos_sim(embedding, fin_avg).numpy()[0][0]]
  legal = [util.cos_sim(embedding, legal_avg).numpy()[0][0]]
  similarity_scores = [bio, fin, legal]
  index = np.argmax(similarity_scores, axis=0)[0]
  return index # returns the index of selected domain 0, 1 or 2 respectively from similarity score list

In [ ]:
accuracy = []
for i in range(len(test_size)):
  question = sub_test[i]['question']
  pred_domain_index = select_domain(question, model, bio_avg, fin_avg, legal_avg)
  if (pred_domain_index == label):
    accuracy.append(1)
  else:
    accuracy.append(0)

#print("Accuracy in domain X: " + str(np.mean(accuracy)))